In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Paths

In [16]:

reshare_fn = '../../data/owid/share-of-electricity-production-from-renewable-sources.csv'

In [15]:
# nz_goals_fn = '../../data/nz_goals/nz_goals.csv'
nz_goals_fn = snakemake.input.nz_goals

#edemand_fn = "../../data/owid/electricity-demand.csv"
edemand_fn = snakemake.input.edemand

# reshare_fn = '../../data/owid/share-of-electricity-production-from-renewable-sources.csv'
reshare_fn = snakemake.input.reshares

NameError: name 'snakemake' is not defined

### Functions to read and prepare demand data

In [8]:
def get_latest_demand(demand_path):

    edemand = pd.read_csv(edemand_fn)

    edemand = edemand[edemand.Code == country]
    # Drop rows where Code is NaN
    edemand = edemand.dropna(subset=['Code'])
    edemand = edemand.rename(columns={'Electricity demand - TWh': 'edemand'})

    # Select latest year and get electricity demand
    edemand = edemand[edemand.Year == edemand.Year.max()].edemand.values[0]

    print(f"The latest available electricity demand of country {country} is {edemand} TWh")

    return edemand

In [14]:
def get_latest_reshare(reshare_path):

    reshare = pd.read_csv(reshare_fn)

    reshare = reshare[reshare.Code == country]
    # Drop rows where Code is NaN
    reshare = reshare.dropna(subset=['Code'])
    reshare = reshare.rename(columns={'Renewables - % electricity': 'reshare'})

    # Select latest year and get electricity demand
    reshare = reshare[reshare.Year == reshare.Year.max()].reshare.values[0]

    print(f"The latest available electricity demand of country {country} is {reshare} %")

    return reshare

### Define variables

In [9]:
# sweep = "year" #["rule", "expansion", "year"]
sweep = snakemake.params.sweep[0]

NameError: name 'snakemake' is not defined

In [10]:
country = "NZL"
year_current = 2024

re_share = get_latest_reshare(reshare_fn) #87.33 # in percent
gen_total = get_latest_demand(edemand_fn) #44.52 # in TWh
ely_eff = 0.62 # in percent (https://github.com/PyPSA/technology-data/blob/master/outputs/costs_2030.csv)

rule = 90 # in percent
rule_sweep = np.array([60, 70, 80, 90]) # in percent

re_exp = np.array([1]) # TWh/year
re_exp_sweep = np.array([0.1,1,2,3]) # TWh/year

year_target = 2035
year_target_sweep = np.array([2030, 2035, 2040, 2045, 2050])

demand_dom_incr = np.array([0, 5, 10, 20]) # in TWh

The latest available electricity demand of country NZL is 44.52 TWh


### Formula

In [11]:
def get_export(sweep="expansion", re_exp_sweep=re_exp_sweep, rule_sweep=rule_sweep, rule=rule, re_exp=re_exp, gen_total=gen_total, ely_eff=ely_eff, year_target=year_target, year_current=year_current, demand_dom_incr=demand_dom_incr):

    if sweep == "expansion":
        dim = re_exp_sweep
        demand_dom_incr_grid, re_exp = np.meshgrid(demand_dom_incr, re_exp_sweep)
    elif sweep == "rule":
        dim = rule_sweep
        demand_dom_incr_grid, rule = np.meshgrid(demand_dom_incr, rule_sweep)
    elif sweep == "year":
        dim = year_target_sweep
        demand_dom_incr_grid, year_target = np.meshgrid(demand_dom_incr, year_target_sweep)

    else:
        raise ValueError("sweep must be one of 'expansion', 'rule', 'year'") 

    export_el = ((re_share-rule)/100 * gen_total) + (re_exp * (year_target-year_current)) - demand_dom_incr_grid

    export = export_el * ely_eff
    export_df = pd.DataFrame(export, columns=demand_dom_incr, index=dim)
    export_df.index.name = sweep
    
    return export_df

In [12]:
export_df = get_export(sweep=sweep)

NameError: name 'sweep' is not defined

### NZ roadmap

In [13]:
nz_goals = pd.read_csv(nz_goals_fn, index_col=0) 
nz_goals *= 33.33e-6 # Convert from tons to TWh hydrogen

NameError: name 'nz_goals_fn' is not defined

### Plot

In [ ]:
fig = plt.figure(facecolor='white', figsize=(6, 5))

ax = fig.add_subplot(1,1,1)
alpha=1

if sweep == "expansion":
    for re_exp_val in np.unique(re_exp_sweep):
        plt.plot(demand_dom_incr, export_df.loc[re_exp_val], label=f'{re_exp_val} TWh/a', color='green', alpha=alpha)
        alpha -= 0.2
        legend_title = 'RE expansion'

elif sweep == "rule":
    for rule_val in np.unique(rule_sweep):
        plt.plot(demand_dom_incr, export_df.loc[rule_val], label=f'{rule_val}% rule', color='green', alpha=alpha)
        alpha -= 0.2
        legend_title = 'Rule'

elif sweep == "year":
    for year_val in np.unique(year_target_sweep):
        plt.plot(demand_dom_incr, export_df.loc[year_val], label=f'{year_val}', color='green', alpha=alpha)
        alpha -= 0.2
        legend_title = 'Year'

# Plot nz hydrogen goals (arrows)
# plt.text(2, nz_goals.loc[2035, "high"], 'NZ 2035 goal', color='red', alpha=1, fontsize=8)
# plt.arrow(2, nz_goals.loc[2035, "high"]+0.27, -0.5,0, head_width=1, head_length=0.5, fc='black', ec='black', alpha=0.5)

# Plot nz hydrogen goals (lines)
for year in nz_goals.index:
    plt.axhline(y=nz_goals.loc[year, "high"], color='red', linestyle='--', alpha=0.5)
    plt.text(0.4, nz_goals.loc[year, "high"]+0.3, f'NZ {year} export target', color='red', alpha=1, fontsize=8)

plt.xlabel('Demand increase in TWh')
plt.ylabel('Export in TWh')
plt.ylim(0, 25)
plt.xlim(0, 20)
plt.grid()
plt.legend(loc='upper right')
# plt.legend(title=legend_title)

fig.savefig(snakemake.output.exportquan, bbox_inches="tight")

plt.show()